# 🧮 Swimming of early larval stages in still water and shear 
This is a notebook interface for using the pyVRS (Python Volume-Rendered Swimmer) model of low-Reynolds number hydrodynamics. It has a simplified interface for entering parameters to investigate stability and movement of early-stage larval morphologies (e.g., eggs, blastulae and gastrulae) approximated by "chimeras" of semi-spheroids. These early larval shapes are constructed by joining two half-spheroids about a common central radius. 

By choosing different spheroid shapes (larger vs. smaller central radii; longer vs. shorter semi-spheroids, etc.) you can construct different surface shapes that resemble the outside surfaces of early stages in larval development. You can assign the main volume inside this surface to have a density that reflects the larval tissue within this surface. 

You can also add inclusions, constructed from two half-spheroids, to reflect the inner structure of early stage larvae. You can assign different densities to model the effects of internal volumes of these inclusions, to reflect whether they are filled with seawater, calcium carbonate, lipid, or other materials.

The resulting shapes do not represent any particular species and stage exactly, but they approximate early stages including eggs, gastrulae and blastulae from a wide diversity of species, and an even wider diversity of hypothetical larval morphologies that do not appear in nature. This makes these shapes good tools for exploring the consequences of size, morphological and physiological features, and properties of the fluid environment such as shear.

This interface has two parts:
- **ChimeraSwim.ipynb** (this notebook) simulates the swimming of early-stage larval shapes using pre-calculated data from files.
- [**ChimeraDesign.ipynb**](./ChimeraDesign.ipynb) enables you to specify new larval shapes and save them into files, that can be loaded and simulated in this notebook. 

## How to use this notebook
This notebook is set up with "cells" that contain either Python code or explanatory text. You can see the cells by the bars and numbers on the left side of the notebook window.

### Executing Jupyter notebook cells
- **To start the model, pull down the `Run` menu and select `Run All Cells`**
- **To execute a code cell, select it and hold down the shift key while typing the `enter` key.**
- **To reset the model to its initial state, pull down the `Run` menu and select `Restart Kernel and Run All Cells`**

### Using graphics output
When the model starts, it creates a larval shape using default parameters. That shape is shown in a separate, resizable graphics window. When the simulation has finished, you can:
- **Click and drag on graphics windows to control the viewing angle.**
- **Click on the floppy disk icon to save graphics windows as images. Use informative names like `default_shape.svg` so that you can tell one image from another later on.**

### Selecting a larval shape
In this notebook, you can select larval shapes from among the "morph.pickle" files:
- [Load a file containing the shape and pre-computed hydrodynamic properties](#section_loadshape)

[In the notebook ChimeraDesign, use input boxes to modify a shape or define a new one, calculate its hydrodynamic properties and save it as a file](./ChimeraDesign)

A good way to start is by loading an existing shape (or use the default shape that is automatically loaded below), and running a few simulations with it.
-  [Run simulations using the current shape](#section_runshape)

When you are familiar with the simulation interface, and have explored the effects of flow on this shape, you can try alternative shapes to see how morphology affects swimming.
You can then save that shape to simulate it in a future session without having to recompute it.
-  [Save a file containing the current shape for use in a future session](#section_saveshape)



In [ ]:
# Set up graphics environment
#%matplotlib inline
#%matplotlib notebook
#import IPython
%matplotlib widget
#%matplotlib ipympl
from matplotlib import pyplot
pyplot.ioff()
#pyplot.ion()
from mpl_interactions import ipyplot as iplt
from mpl_toolkits import mplot3d
from matplotlib.colors import LightSource
# Import modules
import numpy as np
from math import pi
import os
# set up path to submodules
import sys
#sys.path.append('../../../submodules/')
#sys.path.append('../../../submodules/pyVRS')
import pyVRSmorph as mrph
import pyVRSflow as flw
from meshSpheroid import chimeraSpheroid
# Import widget infrastructure
from ipywidgets import interact, interactive, fixed, interact_manual, Output
import ipywidgets as widgets
from IPython import display as idisplay
import pickle
from ipyfilechooser import FileChooser
from copy import deepcopy

In [ ]:
# Create a class and global instances for morphology parameters
global surf_pars, incl1_pars, incl2_pars
class CEpars():
    """
    A simple class to facilitate acquiring and passing chimera ellipsoid
    parameters with interactive_output widgets.
    """
    def __init__(self,D=50.e-6,L1=100.e-6,L2=-40.e-6,d=6e-6,nlevel0=16,nlevel1=12,
                      translate0=0.,translate1=0.,translate2=0.):
        self.D = D
        self.L1 = L1
        self.L2 = L2
        self.Ls = [L1,L2]
        self.d = d
        self.nlevels = [nlevel0,nlevel1]
        self.translate = [translate0,translate1,translate2]

def set_surf_pars(D,L1,L2,d,nlevel0,nlevel1):
    global surf_pars
    surf_pars=CEpars(D=D,L1=L1,L2=L2,d=d,nlevel0=nlevel0,nlevel1=nlevel1)

def set_incl1_pars(D,L1,L2,d,nlevel0,nlevel1,translate0,translate1,translate2):
    global incl1_pars
    incl1_pars=CEpars(D=D,L1=L1,L2=L2,d=d,nlevel0=nlevel0,nlevel1=nlevel1,
                      translate0=translate0,translate1=translate1,translate2=translate2)

def set_incl2_pars(D,L1,L2,d,nlevel0,nlevel1,translate0,translate1,translate2):
    global incl2_pars
    incl2_pars=CEpars(D=D,L1=L1,L2=L2,d=d,nlevel0=nlevel0,nlevel1=nlevel1,
                      translate0=translate0,translate1=translate1,translate2=translate2)

In [ ]:
# Create a class and global instances for simulation parameters
global sim_pars
class SimPars():
    """
    A simple class to facilitate acquiring and passing VRS simulation
    parameters with interactive_output widgets.
    """
    def __init__(self,dudz=0.,dvdz=0.,dwdx=0.,U0=0.,U1=0.,U2=0.,
                 Tmax=20.,cil_speed=0.5*1000*1e-6,
                 phi=pi/3.,theta=-pi/4.,psi=pi):
        self.dudz = dudz
        self.dvdz = dvdz
        self.dwdx = dwdx
        self.U0 = U0
        self.U1 = U1
        self.U2 = U2
        self.Tmax = Tmax
        self.cil_speed = cil_speed
        self.S_fixed = np.asarray([0.,0.,dudz,0.,0.,dvdz,dwdx,0.,0.])
        self.U_const_fixed = np.asarray([U0,U1,U2])
        self.XEinit = np.asarray([0.,0.,0.,phi,theta,psi])

def set_sim_pars(dudz,dvdz,dwdx,Tmax,cil_speed,phi,theta,psi):
    global sim_pars
    sim_pars=SimPars(dudz=dudz,dvdz=dvdz,dwdx=dwdx,
                 Tmax=Tmax,cil_speed=cil_speed,
                 phi=phi,theta=theta,psi=psi)

print('Simulation parameters class defined...')

<hr style="border:10px solid red">

## Loading a larval shape <a id='section_loadshape'></a>

In [ ]:
# Create and display a FileChooser widget
fc_l = FileChooser()
fc_l.filter_pattern = '*.pickle'
fc_l.title = '<b>Choose a filename for this larval morphology:</b>'
fc_l.default_filename = 'newer_morph.pickle'

display(fc_l)
#print(fc_l.selected_path)
#print(fc_l.selected_filename)
#print(fc_l.selected)

:::{figure} #cs_1
:placeholder: ./Images/CS_1.png
:align: left
:::

In [ ]:
# Load the default morphology file, to get a viable scenario for "Run all cells" setup
#M = mrph.Morphology()
with open(fc_l.default_filename, 'rb') as handle:
        #print(f'Loading morphology as {fc_l.default_filename}')
        M = pickle.load(handle)
        print(f'Loaded morphology file {fc_l.default_filename}')

:::{figure} #cs_2
:placeholder: ./Images/CS_2.png
:align: left
:::

In [ ]:
button4 = widgets.Button(description="Load file")
output4 = widgets.Output()

buttonsL = widgets.HBox([button4, output4])
display(buttonsL)

@output4.capture()
def on_button_clicked4(b):
    with open(fc_l.selected, 'rb') as handle:
        #print(f'Loading default morphology as {fc_l.selected}')
        M = pickle.load(handle)
        print(f'Loaded default morphology file {fc_l.selected}')

button4.on_click(on_button_clicked4)

<hr style="border:10px solid red">

## Define the simulation parameters
This cell sets parameters for simulating how the larval shape swims under different flow conditions.

The parameters are:
- $\frac{dU}{dz}$: change of horizontal velocity ($U$) across depth ($z$)
- $\frac{dV}{dz}$: change of horizontal velocity ($V$) across depth ($z$)
- $\frac{dW}{dx}$: change of vertical velocity ($W$) in the $x$ direction
- $\phi$, $\theta$, $\psi$: initial Euler angle
- $T_{max}$: duration of time to simulate
- $V_{cilia}$: velocity of cilia on the larval surface (red: maximum speed; blue: 0 speed)

In [ ]:
# Set simulation parameters
spars_dudz=widgets.FloatText(value=0.,width=10,description = r"$\frac{dU}{dz}$")
spars_dvdz=widgets.FloatText(value=0.,description = r"$\frac{dV}{dz}$")
spars_dwdx=widgets.FloatText(value=0.,description = r"$\frac{dW}{dx}$")
spars_Tmax=widgets.FloatText(value=10,description = r"$T_{max}$")
spars_cil_speed=widgets.FloatText(value=500.e-6,description = r"$V_{cilia}$")
spars_phi=widgets.FloatText(value=pi/3.,description = r"$\phi$")
spars_theta=widgets.FloatText(value=pi/4.,description = r"$\theta$")
spars_psi=widgets.FloatText(value=pi,description = r"$\psi$")

#translate=[0.,0.,40.e-6]

ui0s = widgets.VBox([spars_dudz,spars_phi,spars_Tmax])
ui1s = widgets.VBox([spars_dvdz,spars_theta,spars_cil_speed])
ui2s = widgets.VBox([spars_dwdx,spars_psi])
ui012s = widgets.HBox([ui0s,ui1s,ui2s])


outs = widgets.interactive_output(set_sim_pars,{'dudz':spars_dudz,'dvdz':spars_dvdz,'dwdx':spars_dwdx,'Tmax':spars_Tmax,
                                                'cil_speed':spars_cil_speed,'phi':spars_phi,
                                                'theta':spars_theta,
                                                'psi':spars_psi})
display(ui012s,outs)

:::{figure} #cs_3
:placeholder: ./Images/CS_3.png
:align: left
:::

## To run a simulation <a id='section_runshape'></a>
Hit the **Reset** button to create a new simulation window. 

Hit **Run** to simulate swimming of the larval shape you defined using the flow parameters you defined.

In this window, the left hand side plot shows the trajectory of the larva (blue lines), starting at the position (0,0,0) which is marked with a red dot. The simulated time, and the larval position and velocity are given at the top of the plot.

The right hand side plot shows the orientation of the larval shape. The Euler angles specifying the larva's orientation are given at the top of the plot.

The simulation will plot multiple trajectories in the same plot, to make it easy to compare the effects of different shear rates. You can save the plot using the disk icon at the top of the simulation window. 

- **To run another simulation using the same simulation window, change e.g. the shear or ciliary velocity and click the `Run` button again. Make sure to press the `enter` key after you change parameters.**
- **To run another simulation using a new simulation window, click the `Reset` button and then the `Run` button. The simulation will be plotted in a new graphics window below the previous one.**


In [ ]:
global Sim
button = widgets.Button(description="Reset")
output = widgets.Output()
button2 = widgets.Button(description="Run")
output2 = widgets.Output()

buttons = widgets.HBox([button, output, button2, output2])
display(buttons)

#pyplot.ion()
pyplot.ioff()

@output.capture()
def on_button_clicked(b):
    global Sim, sim_pars
    with pyplot.ioff():
        figS = pyplot.figure(num=58)
    Sim = flw.VRSsim(morph=M,fignum=58)
    #Sim = flw.VRSsim(morph=M,fig=figS,fignum=58)
    #Sim = flw.VRSsim(morph=M,fig=figS)

@output2.capture()
def on_button_clicked2(b):
    global Sim,sim_pars
    Sim.run(XEinit=sim_pars.XEinit,Tmax=sim_pars.Tmax,cil_speed=sim_pars.cil_speed,U_const_fixed=sim_pars.U_const_fixed,S_fixed=sim_pars.S_fixed)

#pyplot.ioff()
button.on_click(on_button_clicked)
#pyplot.ion()
button2.on_click(on_button_clicked2)

:::{figure} #cs_4
:placeholder: ./Images/CS_4.png
:align: left
:::

<hr style="border:10px solid red">